# TUMOR ground station data retrieval

This code is a part of the ground station segment for [TEK5720](https://www.uio.no/studier/emner/matnat/its/TEK5720/) TUMOR (Targeting Ultraviolet Mid-Ozone layer Radiation) mission

Group Blip-Blop:
* Joachim Thomle Karlsen
* Tobias Mellum
* Michał Jan Odorczuk
* Vytenis Orlauskis
* Yawar Seraj

Code made by Michał Jan Odorczuk

This code captures ground station module serial output and saves it to a csv file

Ground station module code is available [here](https://github.com/VytenisO/tumor/blob/main/flightVersion/groundStation.ino)

Ground station receives data transmitted by the [CanSat module](https://github.com/VytenisO/tumor/blob/main/flightVersion/cansat.ino)

## imports

Please install [pyserial](https://pypi.org/project/pyserial/), [pandas](https://pypi.org/project/pandas/) and [matplotlib](https://pypi.org/project/matplotlib/) packages

In [4]:
import serial
from datetime import datetime
import pandas as pd
import serial.tools.list_ports
from matplotlib import pyplot as plt
from IPython.display import clear_output

## Parameters

Please verify the port and baudrate

Cell below lists all possible serial ports

In [5]:
# Get the list of available serial ports
ports = serial.tools.list_ports.comports()

# Iterate through the ports and print their details
for port in ports:
    print(port.device)

port = '/dev/ttyACM3'
baud_rate = 9600
N_UV = 4
MAGNETOMETER_SAMPLES = 10

/dev/ttyACM3
/dev/ttyACM0


## Column names

_i means i-th UV sensor. The indices of the sensors are 0, 1, 2... True column names are then, for example UV_2, AL_0 and so on


| Column name | Description                                                                 |
|-------------|-----------------------------------------------------------------------------|
| UV_i | Ultraviolet counts measured by the i-th UV sensor                                  |
| AL_i | Ambient light counts measured by the i-th UV sensor <br> Value is divided by 256   |
| Mx_i | Normalised magnetic field in x-direction |
| My_i | Normalised magnetic field in y-direction |
| Mz_i | Normalised magnetic field in z-direction |
| time_i| Time in the beginning of the i-th UV sensor frame |
| lon_deg | longitude E in degrees |
| lon_min | longitude E in arcminutes from last full degree |
| lon_sec | longitude E in arcseconds from last full arcminute |
| lat_deg | latitude E in degrees |
| lat_min | latitude E in arcminutes from last full degree |
| lat_sec | latitude E in arcseconds from last full arcminute |
| alt | GPS altitude in meters above sea level |
| time_GPS | timestamp of the GPS readout |
| temperature | external temperature in °C |
| altitude | barometric altitude in meters above sea level


In [6]:
columns = []
for i in range(N_UV):
    columns += [f"UV_{i}", f"AL_{i}", f"Mx_{i}", f"My_{i}", f"Mz_{i}", f"time_{i}"]
columns += ["lon_deg", "lon_min", "lon_sec", "lat_deg", "lat_min", "lat_sec", "alt", "time_GPS", "temperature", "altitude"]

## File header

In [7]:
now = datetime.now()

year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")
second = now.strftime("%S")
microseconds = now.strftime("%f")
filename = f"TUMOR_{year}_{month}_{day}_{hour}_{minute}_{second}.csv"

header = f'''
#              ************************************
#              *****    GLOBAL ATTRIBUTES    ******
#              ************************************
#
#     TITLE                           TUMOR> Ultraviolet Balloon Probe Readings
#     PROJECT                         TUMOR>Targeting Ultraviolet Mid-Ozone layer Radiation
#                                     TEK5720>Space Systems Project>Blip-Blop
#     DISCIPLINE                      Geoscience>Atmospheric Modelling
#     DATA_TYPE                       2.1 second samples
#     DATA_VERSION                    1
#     GENERATED_BY                    Universitetet i Oslo
#     GENERATION_DATE                 {year}{month}{day}
#     LINK_TEXT                       Repository with the code and data
#     LINK_TITLE                      TUMOR project repository
#     HTTP_LINK                       https://github.com/VytenisO/tumor
#     TEXT                            Data obtain from a balloon probe sent as a part of
#                                     TEK5720 course at the University of Oslo
#                                     Balloon was released from Andøya Space Center
#                                     The probe consists of several auxilary sensors like
#                                     GPS, IMU, thermometer and barometer, and four main sensors
#                                     measuring ultraviolet and ambient light detected counts
#                                     distributed evenly in the probe's horizontal plane
#                                     reference:
#                                     https://optoelectronics.liteon.com/upload/download/DS86-2015-0004/LTR-390UV_Final_%20DS_V1%201.pdf
#     MODS                            Initial Release 17/04/24.
#     LOGICAL_FILE_ID                 TUMOR_{year}{month}{day}_V1
#     LOGICAL_SOURCE                  TUMOR
#     LOGICAL_SOURCE_DESCRIPTION      TUMOR 2.1 second UV samples
#     PI_NAME                         M. J. Odorczuk
#     PI_AFFILIATION                  Universitetet i Oslo
#     MISSION_GROUP                   Blip-Blop
#     INSTRUMENT_TYPE                 Ultraviolet sensor
#     TIME_RESOLUTION                 inconsistent, nominally 2.1 second
#     ACKNOWLEDGEMENT                 Please acknowledge the course supervisor
#                                     Anja Kohfeldt from the University of Oslo
#                                     and the course advisor
#                                     Elise Wright Knutsen from the University of Oslo
#     RULES_OF_USE                    Free to use, source acknowledge required
#
#              ************************************
#              ****  RECORD VARYING VARIABLES  ****
#              ************************************
#
#  1. Ultraviolet frames {", ".join([f"{i}" for i in range(N_UV - 1)])} and {N_UV - 1}
#       1a. Ultraviolet counts on the sensor
#           NOTES:  Reference: https://optoelectronics.liteon.com/upload/download/DS86-2015-0004/LTR-390UV_Final_%20DS_V1%201.pdf
#                   Gain set to 18, resolution set to 16 bit, integration time is 25 ms
#                   Values may range from 0 to 65535, expected no larger than 3000
#       1b. Ambient light counts on the sensor
#           NOTES:  Reference: https://optoelectronics.liteon.com/upload/download/DS86-2015-0004/LTR-390UV_Final_%20DS_V1%201.pdf
#                   Gain set to 18, resolution set to 16 bit, integration time is 25 ms
#                   Values may range from 0 to 255
#                   Counts are divided by 256
#       1c. Magnetic field on the probe in x-direction, normalized to total magnitude equal to 1
#       1d. Magnetic field on the probe in y-direction, normalized to total magnitude equal to 1
#       1e. Magnetic field on the probe in z-direction, normalized to total magnitude equal to 1
#           NOTES:  Magnetic field is an average of {MAGNETOMETER_SAMPLES} samples measured right before 1a and another {MAGNETOMETER_SAMPLES} samples after 1b
#                   Values are interpolated as an average of all the samples
#       1f. Timestamp at the beginning of the frame
#           NOTES:  First {MAGNETOMETER_SAMPLES} magnetometer samples are obtained up to {MAGNETOMETER_SAMPLES}ms from the timestamp
#                   Ultraviolet counts are sampled in the window between {max(MAGNETOMETER_SAMPLES + 1, 26) - 25} to {max(MAGNETOMETER_SAMPLES + 1, 26)}ms from the timestamp
#                   Ambient light counts are sampled in the window between {max(MAGNETOMETER_SAMPLES + 1, 26) + 1} and {max(MAGNETOMETER_SAMPLES + 1, 26) + 26}ms from the timestamp
#                   Last {MAGNETOMETER_SAMPLES} magnetometer samples are obtained {max(MAGNETOMETER_SAMPLES + 1, 26) + 26 + MAGNETOMETER_SAMPLES}ms from the timestamp
#                   Timestamp given in the format hh:mm:ss.sss
#       NOTES: All the points 1a-1f are repeated four times for each sensor
#  2. GPS frame
#       2a. Longitude of the probe in degree east
#       2b. Longitude of the probe in arcminutes east from the last full degree
#       2c. Longitude of the probe in arcseconds east from the last full arcminute
#       2d. Latitude of the probe in degree east
#       2e. Latitude of the probe in arcminutes east from the last full degree
#       2f. Latitude of the probe in arcseconds east from the last full arcminute
#       2g. Altitude in meters above sea level
#       2h. Timestamp of the GPS data frame
#       NOTES:  Received GPS data is sent in clipped compressed values thus overflow errors can occur
#               Overflow spherical coordinates error occurs for every 18°12'18''
#               Overflow altitudinal error occurs for every 65 536 m
#               Overflow temporal error occurs for every 18 hours, 12 minutes and 18 seconds
#               Expected launch coordinates are 69°19' N, 16°7' E
#               Expected launch time is {day}-{month}-{year} between 12:45:00 - 15:00:00 UTC
#               Expected altitude is from 0 to 30 000 m above sea level
#  9. Barometric altitude
#       NOTES:  In meters above sea level based on CanSat book from Andøya Space Education
#               Reference: https://learn.andoyaspace.no/ebook/the-cansat-book/common/getting-started/using-the-sensors/altitude-calculations/
#               Overflow altitudinal error occurs for every 65 536 m
#               Expected altitude is from 0 to 30 000 m above sea level
# 10. External temperature
#       NOTES:  Given in degrees Celsius
#               Overflow error occurs for every 128 degrees
#               Values ranging from -84 to +43.5
#               Expected temperature is from -70 to +30
#
#              ************************
#              ****  COLUMN NAMES  ****
#              ************************
#
# {" ".join([f"UV_{i}   AL_{i}   Mx_{i}   My_{i}   Mz_{i}   time_{i}      " for i in range(N_UV)])} lon_deg lon_min lon_sec lat_deg lat_min lat_sec alt time_GPS temperature altitude
# {" ".join([ "counts counts #      #      #      hh:mm:ss.sss" for i in range(N_UV)])            } °E      'E      ''E     °N      'N      ''N         hh:mm:ss °C          m
#
#
'''


## GUI functionality

Set GUI_FLAG to True if you want the script to display the plots

Set GUI_FLAG to False if you do not want the script to display the plots

In [16]:
uv = [[] for i in range(N_UV)]
al = [[] for i in range(N_UV)]
mx = [[] for i in range(N_UV)]
my = [[] for i in range(N_UV)]
mz = [[] for i in range(N_UV)]
time = [[] for i in range(N_UV)]
lon = []
lat = []
alt = []
time_gps = []
alt_bar = []
temperature = []

GUI_FLAG = True

sensor_time_offset = 0
gps_time_offset = 0

def displayGUI(frame):
    global sensor_time_offset
    global gps_time_offset
    for i in range(N_UV):
        uv[i].append(int(frame[f"UV_{i}"].iloc[0]))
        al[i].append(int(frame[f"AL_{i}"].iloc[0]))
        mx[i].append(float(frame[f"Mx_{i}"].iloc[0]))
        my[i].append(float(frame[f"My_{i}"].iloc[0]))
        mz[i].append(float(frame[f"Mz_{i}"].iloc[0]))
        time[i].append(int(frame[f"time_{i}"].iloc[0]) + sensor_time_offset)
    lon.append(int(frame["lon_deg"].iloc[0]) + int(frame["lon_min"].iloc[0]) / 60 + int(frame["lon_sec"].iloc[0]) / 3600)
    lat.append(int(frame["lat_deg"].iloc[0]) + int(frame["lat_min"].iloc[0]) / 60 + int(frame["lat_sec"].iloc[0]) / 3600)
    alt.append(int(frame["alt"].iloc[0]))
    time_gps.append(int(frame[-1]['time_GPS'].iloc[0]) + gps_time_offset)
    alt_bar.append(int(frame["altitude"].iloc[0]))
    temperature.append(int(frame["temperature"].iloc[0]))

    # Create a figure and subplots
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

    axes[0, 0].set_title('UV')
    axes[0, 1].set_title('AL')
    axes[0, 2].set_title('Magnetic field')
    for i in range(N_UV):
        # Plot the data on the subplots
        axes[0, 0].plot(time[i], uv[i])

        axes[0, 1].plot(time[i], al[i])

        axes[0, 2].plot(time[i], mx[i])
        axes[0, 2].plot(time[i], my[i])
        axes[0, 2].plot(time[i], mz[i])

    axes[1, 0].plot(lon, lat)
    axes[1, 0].set_title('coordinates')

    axes[1, 1].plot(time_gps, alt)
    axes[1, 1].plot(time_gps, alt_bar)
    axes[1, 1].set_title('Altitude')

    axes[1, 2].plot(gps_time, temperature)
    axes[1, 2].set_title('Temperature')

    # Adjust layout for better appearance
    plt.tight_layout()
    clear_output(wait=True)
    # Display the subplots
    plt.show()

## Fetch and save to a file

Please rerun the header to get correct time, header and file name

In [17]:
# store frames in case of further analysis
frames = []


with open(filename, 'w') as file:
    file.write(header)
    file.write(",".join(columns) + "\n")

# Open the serial port
ser = serial.Serial(port, baud_rate)

gps_time_offset = int(second) + int(minute) * 60 + int(hour) * 3600
gps_time_offset -= gps_time_offset % (2 ** 16)

sensor_time_offset = (gps_time_offset * 1000) % (2 ** 16)

last_gps_time = 0

# Read and print the serial output
while True:
    dataline = ser.readline().decode('utf-8').strip()  # Convert bytes to string
    frames.append(pd.DataFrame([dataline.split(',')], columns=columns))
    new_gps_time = int(frames[-1]['time_GPS'].iloc[0])
    if (new_gps_time > 0 and new_gps_time < last_gps_time):
        gps_time_offset += 2 ** 16
    gps_time = new_gps_time + gps_time_offset
    displayGUI(frames[-1])
    frames[-1]['time_GPS'] = f"{gps_time // 3600}:{(gps_time // 60) % 60}:{gps_time % 60}"
    if new_gps_time > 0:
        last_sensor_time = int(frames[-1][f"time_{N_UV - 1}"].iloc[0])
        sensor_time_offset = gps_time_offset * 1000 - last_sensor_time
    for i in range(N_UV):
        sensor_time = int(frames[-1][f"time_{i}"].iloc[0]) + sensor_time_offset
        frames[-1][f"time_{i}"] = f"{sensor_time // 3_600_000}:{(sensor_time // 60_000) % 60}:{(sensor_time // 1000) % 60}.{(sensor_time) % 1000}"

    frames[-1].to_csv(filename, mode='a', index = False, header = False)


KeyError: -1